In [1]:
TLC_URL = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
EVENT_URL = 'https://data.cityofnewyork.us/api/views/6v4b-5gp4/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B'

LANDING_DATA = '../data/landing/'
ALL_SOURCES = ('yellow', 'green', 'fhvhv')


In [5]:
import os

# from the current `proj1` directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('landing', 'raw', 'curated', 'analysis'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

output_relative_dir = '../data/landing/'
for target_dir in ALL_SOURCES: 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [6]:
from urllib.request import urlretrieve

def retrieve_data(source: str, year: str, month: str) -> None:
    # generate url
    get_url = f'{TLC_URL+source+"_tripdata_"}{year}-{month}.parquet'
    # generate output location and filename
    get_path = f'{LANDING_DATA+source+"/"}/{year}-{month}.parquet'
    # download
    urlretrieve(get_url, get_path)


In [8]:
import itertools

# take all data from May 2020 - May 2023
years = ['2020', '2021', '2022', '2023']
start_year = '2020'; start_month = 5
end_year = '2023'; end_month = 5
months = range(1, 13)

all_months = itertools.product(years, months)
for year_month in all_months:
    year, month = year_month
    if not((year == start_year and month < start_month) | (year == end_year and month > end_month)):
        # 0-fill i.e 1 -> 01, 2 -> 02, etc
        month = str(month).zfill(2) 
        print(f"Begin year {year} month {month}")
        for source in ALL_SOURCES:
            retrieve_data(source, year, month)        
        print(f"Completed year {year} month {month}")

Begin year 2022 month 01
Completed year 2022 month 01
Begin year 2022 month 02
Completed year 2022 month 02
Begin year 2022 month 03
Completed year 2022 month 03
Begin year 2022 month 04
Completed year 2022 month 04
Begin year 2022 month 05
Completed year 2022 month 05
Begin year 2022 month 06
Completed year 2022 month 06
Begin year 2022 month 07
Completed year 2022 month 07
Begin year 2022 month 08
Completed year 2022 month 08
Begin year 2022 month 09
Completed year 2022 month 09
Begin year 2022 month 10
Completed year 2022 month 10
Begin year 2022 month 11
Completed year 2022 month 11
Begin year 2022 month 12
Completed year 2022 month 12


In [3]:
from urllib.request import urlretrieve

# generate output location and filename
get_path = f'{LANDING_DATA}Parks_Special_Events.csv'
# download
urlretrieve(EVENT_URL, get_path)

('../data/landing/Parks_Special_Events.csv',
 <http.client.HTTPMessage at 0x7faccaf55de0>)

In [4]:
import pandas as pd

event_data = pd.read_csv(f'{LANDING_DATA}Parks_Special_Events.csv')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 6, saw 3


In [ ]:
import os
from datetime import date, datetime, timedelta

import requests


def fetch(station_id):
    """Download data we are interested in!"""
    localfn = f"{station_id}.csv"
    if os.path.isfile(localfn):
        print(f"- Cowardly refusing to over-write existing file: {localfn}")
        return
    print(f"+ Downloading for {station_id}")
    enddt = date.today() + timedelta(days=2)
    uri = (
        "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
        f"station={station_id}&data=tmpc&year1=2021&month1=5&day1=1&"
        f"year2=2023&month2=5&day2=31&"
        "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&"
        "direct=yes&report_type=3"
    )
    res = requests.get(uri, timeout=300)
    with open(localfn, "w", encoding="utf-8") as fh:
        fh.write(res.text)


def main():
    """Main loop."""
    # Step 1: Fetch global METAR geojson metadata
    # https://mesonet.agron.iastate.edu/sites/networks.php
    req = requests.get(
        "http://mesonet.agron.iastate.edu/geojson/network/AZOS.geojson",
        timeout=60,
    )
    geojson = req.json()
    for feature in geojson["features"]:
        station_id = feature["id"]
        props = feature["properties"]
        # We want stations with data to today (archive_end is null)
        if props["archive_end"] is None:
            continue
        # We want stations with data to at least 1943
        if props["archive_begin"] is None:
            continue
        archive_begin = datetime.strptime(props["archive_begin"], "%Y-%m-%d")
        if archive_begin.year > 1943:
            continue
        # Horray, fetch data
        fetch(station_id)

